In [7]:
import glob
import pandas as pd
import SimpleITK as sitk
import cv2
from albumentations import Resize
import skimage.measure
import os
from tqdm import tqdm

In [8]:
%cd C:/Users/dhkim/Desktop/directory/k-ium

C:\Users\dhkim\Desktop\directory\k-ium


In [9]:
patient_list_forder = glob.glob('data/2023_k_ium_composition/patient_list/*')

patient_img = glob.glob(patient_list_forder[0]+'/*')
patient_img

['data/2023_k_ium_composition/patient_list\\patient_1001\\1001LI-A.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LI-B.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LV-A.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LV-B.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RI-A.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RI-B.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RV-A.jpg',
 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RV-B.jpg']

In [10]:
def save_to_off(filepath, verts, faces):
    with open(filepath, "w") as file:
        file.write("OFF\n")
        file.write(f"{len(verts)} {len(faces)} 0\n")
        for vert in verts:
            file.write(" ".join([str(v) for v in vert]) + "\n")
        for face in faces:
            file.write("3 " + " ".join([str(f) for f in face]) + "\n")

In [11]:
import SimpleITK as sitk
import numpy as np

# List of file paths for the 8 2D images
patient_list_forder = glob.glob('data/2023_k_ium_composition/patient_list/*')

for i in tqdm(range(len(patient_list_forder))):
    patient_img = glob.glob(patient_list_forder[i]+'/*')
#     print(len(patient_img))
#     print("i :", i)
    
    image_paths = patient_img
    
#     print("image_paths")
#     print(image_paths)
    
    
    # Load the 2D images
    image_stack = []
    for path in image_paths:
        image = sitk.ReadImage(path)
        
        # 이미지 크기를 720 x 720으로 줄입니다.
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(image.GetSpacing())
        resampler.SetSize((720, 720))
        resampler.SetOutputDirection(image.GetDirection())
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetTransform(sitk.Transform())
        resampler.SetInterpolator(sitk.sitkLinear)

        resampled_image = resampler.Execute(image)
        
        image_stack.append(resampled_image)

#     print("!!!!!!!!!!!!!!")
#     print(len(image_stack))

    # Convert the image stack to a 3D volume
    volume = sitk.JoinSeries(image_stack)
    # print("volume :", volume)

    # Get the size and spacing information from one of the images
    size = image_stack[0].GetSize()
    spacing = image_stack[0].GetSpacing()
    spacing = (spacing[0], spacing[1], 1.0)  # Assuming a spacing of 1.0 for the z-axis
    print("spacing :", spacing)

    # Set the size and spacing for the 3D volume
    new_size = (size[0], size[1], len(image_stack))

    # Create a reference image with the desired size and spacing
    reference_image = sitk.Image(new_size, volume.GetPixelIDValue())
    reference_image.SetSpacing(spacing)

    # Resample the volume to match the reference image
    resampled_volume = sitk.Resample(volume, reference_image)

    # Convert the 3D volume to a numpy array
    volume_array = sitk.GetArrayFromImage(resampled_volume)

    # Print the shape of the volume array
#     print("Volume shape:", volume_array.shape)
    
    
    # 3d 데이터를 .off 로 저장하기
    mean_volume = np.mean(volume_array, axis=-1)
#     print("Mean volume shape:", mean_volume.shape)  # (8, 720, 720)
    
    
    threshold = np.max(mean_volume) * 0.5  # 적절한 임계값 설정
    verts, faces, *_ = skimage.measure.marching_cubes(mean_volume, level=threshold)
    
    pat = image_paths[0]
    pat_num = pat[49:53]
    pat_num
    os.makedirs(f"C:/Users/dhkim/Desktop/directory/k-ium/data/3d_patient_list/{pat_num}_pat", exist_ok=True)
    your_filepath = f"C:/Users/dhkim/Desktop/directory/k-ium/data/3d_patient_list/{pat_num}_pat/{pat_num}.off"
    
    save_to_off(your_filepath, verts, faces)
    
    

  0%|                                                                                         | 0/1127 [00:00<?, ?it/s]

8
i : 0
image_paths
['data/2023_k_ium_composition/patient_list\\patient_1001\\1001LI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001LV-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1001\\1001RV-B.jpg']
pppppppppp
1001
!!!!!!!!!!!!!!
8
spacing : (1.0, 1.0, 1.0)
Volume shape: (8, 720, 720, 3)
Mean volume shape: (8, 720, 720)


  0%|                                                                               | 1/1127 [00:05<1:40:28,  5.35s/it]

8
i : 1
image_paths
['data/2023_k_ium_composition/patient_list\\patient_1002\\1002LI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002LI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002LV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002LV-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002RI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002RI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002RV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_1002\\1002RV-B.jpg']
pppppppppp
1002
!!!!!!!!!!!!!!
8
spacing : (1.0, 1.0, 1.0)
Volume shape: (8, 720, 720, 3)
Mean volume shape: (8, 720, 720)


  0%|▏                                                                              | 2/1127 [00:10<1:40:45,  5.37s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▊| 1125/1127 [1:27:36<00:09,  4.55s/it]

8
i : 1125
image_paths
['data/2023_k_ium_composition/patient_list\\patient_2610\\2610LI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610LI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610LV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610LV-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610RI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610RI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610RV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2610\\2610RV-B.jpg']
pppppppppp
2610
!!!!!!!!!!!!!!
8
spacing : (1.0, 1.0, 1.0)
Volume shape: (8, 720, 720, 3)
Mean volume shape: (8, 720, 720)


100%|███████████████████████████████████████████████████████████████████████████▉| 1126/1127 [1:27:40<00:04,  4.34s/it]

8
i : 1126
image_paths
['data/2023_k_ium_composition/patient_list\\patient_2611\\2611LI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611LI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611LV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611LV-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611RI-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611RI-B.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611RV-A.jpg', 'data/2023_k_ium_composition/patient_list\\patient_2611\\2611RV-B.jpg']
pppppppppp
2611
!!!!!!!!!!!!!!
8
spacing : (1.0, 1.0, 1.0)
Volume shape: (8, 720, 720, 3)
Mean volume shape: (8, 720, 720)


100%|████████████████████████████████████████████████████████████████████████████| 1127/1127 [1:27:44<00:00,  4.67s/it]


In [12]:
# 그림한번 보기
from tqdm import tqdm

In [13]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Choose how many slices along z-axis to show
# num_slices_to_show = 8

# # Create the 3D figure
# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(111, projection='3d')

# for k in tqdm(range(num_slices_to_show)):
#     img_array = volume_array[k]
    
#     # If the images have 3 channels, reduce them to a single channel by taking the average
#     if img_array.shape[-1] == 3:
#         img_array = img_array.mean(axis=-1)
    
#     # Normalize the image for better visualization
#     img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())
    
#     # Define the X, Y, and Z coordinates of the image
#     x = np.arange(img_array.shape[1]) + 0.5
#     y = np.arange(img_array.shape[0]) + 0.5
#     z = k
    
#     X, Y = np.meshgrid(x, y)
#     Z = np.full_like(X, z)
    
#     # Display the image slice at the specified Z coordinate
#     ax.plot_surface(X, Y, Z, rstride=1, cstride=1, facecolors=plt.cm.gray(img_array), shade=False)

# # Set axis labels
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')

# # Set view angle
# ax.view_init(elev=30, azim=45)

# plt.show()
